## Reactive Covariance - Project in Python 

### I - Leverage Effect in Financial Markets (Paper 1) 


In [ ]:
# Import relevant libraries

import matplotlib.pyplot as plt
import pandas as pd
import os
import fnmatch
import random
import scipy as sp
from scipy.optimize import curve_fit
import numpy as np

In [ ]:
## Cette partie permet de lister tous les Stocks et ETFs dans les dossiers pour créer les dictionnaires automatiquement contenant les stocks et ETFs

## Pour les Stocks

directoryStocks = "Data\Stocks"

numberStocks = len(fnmatch.filter(os.listdir(directoryStocks), '*.txt')) # Cela donne le nombre total de stocks dans la folder
full_stock_dictionnary = {} # Creation d'un dictionnaire vide pour les stocks

j = 1
for i in os.listdir(directoryStocks):
    full_stock_dictionnary["Ticker" + str(j)] = i  
    j= j+1

## Pour les Indices

directoryIndex = "Data\ETFs"

numberIndex = len(fnmatch.filter(os.listdir(directoryIndex), '*.txt')) # Cela donne le nombre total de ETFs dans la folder
full_index_dictionnary = {} # Creation d'un dictionnaire vide pour les indices

#index_dictionnary = {i for i in os.listdir(directoryIndex)} ## Remplacer par numberStocks pour looper sur tous les stocks du dossier 

j = 1
for i in os.listdir(directoryIndex):
    full_index_dictionnary["Ticker" + str(j)] = i  
    j= j+1

print(full_stock_dictionnary)
print(full_index_dictionnary)


In [ ]:
# Choisissons un subset aléatoire du dictionnaire de stocks et d'indices pour calculer le leverage

index_dictionnary = {}

for j in range(1,20):
    index_dictionnary["Ticker" + str(j)] = full_index_dictionnary[random.choice(list(full_index_dictionnary.keys()))]
    j= j+1

print(index_dictionnary)

stock_dictionnary = {}

for j in range(1,10):
    stock_dictionnary["Ticker" + str(j)] = full_stock_dictionnary[random.choice(list(full_stock_dictionnary.keys()))]
    j= j+1

print(stock_dictionnary)

In [ ]:
## Le but de cette classe est de regrouper les méthodes permettant le calcul du leverage
## L'idée est ensuite de pouvoir calculer rapidement le leverage en loopant sur des centaines de stocks et indices 

class ComputeLeverage: 
    
    def __init__(self): ## Méthode du constructeur
        self.ticker = Ticker ## on construit l'objet prenant comme parametre le ticker du stock
        self.security = Security ## Permet de préciser s'il s'agit d'un indice ou d'un stock pour choisir le bon répertoire
        self.df = pd.read_csv(os.path.join("Data\\", self.security, self.ticker),delimiter=',',usecols=['Date','Open','High','Low','Close']) # This is an S&P 500 ETF
        # On importe les données dans la construction de l'objet. La dataframe "df" sera utilisee par les autres méthodes de la classe
        
    def sortData(self):
        self.df = self.df.sort_values("Date") # sort the data by date
        return self.df.head()
    
    def deltaTau(self,tau): # Cette fonction calcule la variation du prix entre t et t + tau (delta x(t+tau)) 
        return (self.df["Close"].shift(-tau) - self.df["Close"].shift(-tau+1))/self.df["Close"].shift(-tau+1)
    
    def zHat(self,tau,T): # Cette fonction calcule Z chapeau
        self.df["Interm_Z"] = (self.deltaTau(tau)**2)  # Calcule l'intérieur de la somme 
        return ((1/T)*(self.df["Interm_Z"].sum())**2) # Retourne la somme des carrés normalisé par T
        
    def lHat(self,tau,T): # C
        self.df["Interm_Prod_Delta"]  = (self.deltaTau(tau))**2*self.deltaTau(0)
        return ((1/(self.zHat(tau,T)*(T-tau)))*self.df["Interm_Prod_Delta"].sum())
    
    def PlotLeverage(self):
        T = self.df.shape[0] # size of our sample
        result_df = pd.DataFrame([])
        for tau in range(0,250): ## specify the max range for the time horizon 
             result_df = result_df.append(pd.DataFrame({"Tau":tau,'L_Hat': self.lHat(tau,T)}, index=[0]), ignore_index=True)
                
        return result_df.plot.scatter(x = "Tau", y = "L_Hat" )
    
    ## C'est exactement la même méthode qu'au dessus. La seule différence est qu'elle retourne le résultat sous forme de tableau et non de graph
    def ReturnLeverage(self):
        T = self.df.shape[0] # size of our sample
        result_df = pd.DataFrame([])
        for tau in range(0,250): ## specify the max range for the time horizon 
            result_df = result_df.append(pd.DataFrame({"Tau":tau,self.ticker: self.lHat(tau,T)}, index=[0]), ignore_index=True)
        return result_df

In [ ]:
Tau = 200  ## On spécifie ici l'horizon de temps qui sera calculé au sein de la classe 
Security = "ETFs" ## Select "ETFs" ou "Stocks" pour calculer le leverage des indices / stocks. 
                  ## Cela change le répertoire d'importation des donnees dans la création de la classe

leverage_df = pd.DataFrame({"Tau":range(0,Tau)}) # Création de la première colonne comportant les valeurs de Tau

if (Security == "ETFs"):
    my_dictionnary = index_dictionnary
else:
    my_dictionnary = stock_dictionnary


for key in my_dictionnary:
    Ticker = my_dictionnary.get(key) ## définition du ticker pour la création de l'objet
    myStock = ComputeLeverage() ## Creation de l'objet 
    #myStock.PlotLeverage() ## On appelle la methode qui trace le graph - décommenter pour montrer les graphs
    leverage_df = pd.concat([leverage_df,myStock.ReturnLeverage()[Ticker]], axis=1) # Concaténation du leverage pour chaque stock dans la dataframe Leverage_df

print(leverage_df)

In [ ]:
# On cree une colonne qui prend la moyenne du leverage pour tous les stocks pour chaque maturité Tau
mean_leverage_df = pd.DataFrame({"Tau":range(0,Tau)}) # Création de la première colonne comportant les valeurs de Tau
mean_leverage_df["Mean Leverage"] = leverage_df.drop(["Tau"],axis=1).mean(axis=1) # Ne pas oublier de moyenner en enlevant la colonne Tau!

print(mean_leverage_df)
mean_leverage_df.plot.scatter(x = "Tau", y = "Mean Leverage" ) # Plot pour voir le résultat 

#x= mean_leverage_df["Tau"]
#y= mean_leverage_df["Mean Leverage"]
#sp.optimize.curve_fit(lambda t,a,b: a*np.exp(b*t),  x,  y)
#plt.plot(sp.optimize.curve_fit(lambda t,a,b: a*np.exp(b*t),  x,  y))



In [ ]:
def func(x, a, b, c):
  # return (a*b)
  return a * np.exp(-b*x) +c

x= mean_leverage_df["Tau"]
y= mean_leverage_df["Mean Leverage"]

popt, pcov = curve_fit(func, x, y,p0=(0, 1e-4, 0))

plt.figure()
plt.plot(x, y, 'ko', label="Original Noised Data")
plt.plot(x, func(x, *popt), 'r-', label="Fitted Curve")
plt.legend()
plt.show()